You should remove the `raise` exceptions below and insert your code in their place. The cells which say `DO NOT CHANGE THE CONTENT OF THIS CELL` are there to help you, if they fail, it's probably an indication of the fact that your code is wrong. You should not change their content - if you change them to make them correspond to what your program is producing, you will still not get the marks.

If you encounter an error while running your notebook that doesn't appear to be connected to RDDs (such as missing `imp`), you should check that you've run the initialization cells since you've started your latest cluster.

Before you turn your solution in, make sure everything runs as expected. With an attached cluster, you should **Clear State and Results** (under the **Clear** dropdown menu) and then click on the **Run all** icon. This runs all cells in the notebook from new. You should only submit this notebook if all cells run.

This homework is to be completed on your own. By the act of following these instructions and handing your work in, it is deemed that you have read and understand the rules on plagiarism as written in your student handbook.

# Scrubbing data

A common part of the ETL process is data scrubbing. This homework asks you to process data in order to get it into a standardized format for later processing.

The file `devicestatus.txt.zip` is available from Blackboard. This file contains data collected from mobile devices on a network, including device ID, current status, location and so on. Because the company previously acquired other mobile provider's networks, the data from different subnetworks has a different format. Note that the records in this file have different field delimiters: some use commas, some use pipes (|), and so on. 

This notebook will guide you through the steps of scrubbing this dataset. Follow the instructions carefully. In general, every time you execute a step, there is a check that tests whether you carried out that step correctly. You should not move on until a step is passing the test. All your processing should be done in this notebook. It is assumed that you have uncompressed the `devicestatus.txt` file and have placed it in `/FileStore/` on DBFS using the steps described in the labs this week.

The first two cells perform some checks: first one checks that you are running this notebook on Databricks, and the second cell ensures that your file is starting off in `/FileStore/devicestatus.txt` You should not (try to) change either of these cells.

#### Checking uploaded file

In [0]:
dbutils.fs.ls('/FileStore/tables/')

Out[1]: [FileInfo(path='dbfs:/FileStore/tables/accounts.zip', name='accounts.zip', size=5297592),
 FileInfo(path='dbfs:/FileStore/tables/devicestatus.zip', name='devicestatus.zip', size=23873574),
 FileInfo(path='dbfs:/FileStore/tables/json_activations.zip', name='json_activations.zip', size=8411369),
 FileInfo(path='dbfs:/FileStore/tables/logs.zip', name='logs.zip', size=18168065)]

#### Copying uploaded file to temporary storage

In [0]:
dbutils.fs.cp("dbfs:/FileStore/tables/devicestatus.zip", "file:/tmp")

Out[2]: True

#### Checking devicestatus.zip file into local temp folder

In [0]:
dbutils.fs.ls("file:/tmp")

Out[3]: [FileInfo(path='file:/tmp/hsperfdata_root/', name='hsperfdata_root/', size=4096),
 FileInfo(path='file:/tmp/Rtmp1Ux14c/', name='Rtmp1Ux14c/', size=4096),
 FileInfo(path='file:/tmp/systemd-private-fb4144ef634f40cd866a02d896393e32-systemd-resolved.service-RQqtgi/', name='systemd-private-fb4144ef634f40cd866a02d896393e32-systemd-resolved.service-RQqtgi/', size=4096),
 FileInfo(path='file:/tmp/tmp.bx4TbXk6dh', name='tmp.bx4TbXk6dh', size=0),
 FileInfo(path='file:/tmp/.Test-unix/', name='.Test-unix/', size=4096),
 FileInfo(path='file:/tmp/.X11-unix/', name='.X11-unix/', size=4096),
 FileInfo(path='file:/tmp/chauffeur-env.sh', name='chauffeur-env.sh', size=156),
 FileInfo(path='file:/tmp/driver-env.sh', name='driver-env.sh', size=3267),
 FileInfo(path='file:/tmp/devicestatus.zip', name='devicestatus.zip', size=23873574),
 FileInfo(path='file:/tmp/custom-spark.conf', name='custom-spark.conf', size=216),
 FileInfo(path='file:/tmp/systemd-private-fb4144ef634f40cd866a02d896393e32-apache2.

#### Unzipping devicestatus.zip file into local temp folder

In [0]:
%sh
unzip -d /tmp/ /tmp/devicestatus.zip

Archive:  /tmp/devicestatus.zip
  inflating: /tmp/devicestatus.txt   


#### Moving the txt file to the dbfs FileStore

In [0]:
dbutils.fs.mv("file:/tmp/devicestatus.txt", "dbfs:/FileStore", True)

Out[5]: True

#### Checking file moved or not !

In [0]:
dbutils.fs.ls("/FileStore")

Out[6]: [FileInfo(path='dbfs:/FileStore/account_models/', name='account_models/', size=0),
 FileInfo(path='dbfs:/FileStore/accounts/', name='accounts/', size=0),
 FileInfo(path='dbfs:/FileStore/activations/', name='activations/', size=0),
 FileInfo(path='dbfs:/FileStore/devicestatus.txt', name='devicestatus.txt', size=64124001),
 FileInfo(path='dbfs:/FileStore/devicestatus_etl/', name='devicestatus_etl/', size=0),
 FileInfo(path='dbfs:/FileStore/landsat data.txt', name='landsat data.txt', size=7785),
 FileInfo(path='dbfs:/FileStore/landsat_data.txt', name='landsat_data.txt', size=7785),
 FileInfo(path='dbfs:/FileStore/logs/', name='logs/', size=0),
 FileInfo(path='dbfs:/FileStore/tables/', name='tables/', size=0)]

In [0]:
# DO NOT CHANGE THE CONTENT OF THIS CELL
import sys
if not 'dbruntime.dbutils' in sys.modules.keys():
    import pyspark
    sc = pyspark.SparkContext()
    print("Unless you're grading this homework, you should be running this on Databricks.")

In [0]:
# DO NOT CHANGE THE CONTENT OF THIS CELL
if 'dbruntime.dbutils' in sys.modules.keys():
    try:
        dbutils.fs.ls("/FileStore/devicestatus.txt")
    except:
        assert False, "It is assumed that you've put your (unzipped) devicestatus.txt file in the DBFS /FileStore"
else:    
    import os
    assert os.path.exists("/FileStore/devicestatus.txt") == True, "It is assumed that you've put your (unziped) devicestatus.txt file in the DBFS /FileStore"

#### Working with myRDD

Load the `devicestatus.txt` dataset into a variable called `myRDD`.

In [0]:
# Read the devicestatus.txt file into an RDD. Your answer should have the following format (without comment tag)
myRDD = sc.textFile("dbfs:/FileStore/devicestatus.txt")


In [0]:
# DO NOT CHANGE THE CONTENT OF THIS CELL
assert myRDD.count() == 459540, "It doesn't look like you've read in your data correctly"

#### Working with splitRDD

Determine which delimiter(s) to use and divide up each line of the RDD into a list of its fields. Your result should be in a variale called `splitRDD`.

In [0]:
# The format of your answer should be
import re
pattern = r"[|]|[/]|[,]"
splitRDD = myRDD.map(lambda x: re.split(pattern, x)) \
                .map(lambda fields: (fields))

##### Checking splitRDD output

In [0]:
splitRDD.take(2)

Out[12]: [['2014-03-15:10:10:20',
  'Sorrento F41L',
  '8cc3b47e-bd01-4482-b500-28f2342679af',
  '7',
  '24',
  '39',
  'enabled',
  'disabled',
  'connected',
  '55',
  '67',
  '12',
  '33.6894754264',
  '-117.543308253'],
 ['2014-03-15:10:10:20',
  'MeeToo 1.0',
  'ef8c7564-0a1a-4650-a655-c8bbd5f8f943',
  '0',
  '31',
  '63',
  '70',
  '39',
  '27',
  'enabled',
  'enabled',
  'enabled',
  '37.4321088904',
  '-121.485029632']]

In [0]:
# DO NOT CHANGE THE CONTENT OF THIS CELL
assert splitRDD.sortBy(lambda x: x[0]).take(2) == [[u'2014-03-15:10:10:20',
  u'Sorrento F41L',
  u'8cc3b47e-bd01-4482-b500-28f2342679af',
  u'7',
  u'24',
  u'39',
  u'enabled',
  u'disabled',
  u'connected',
  u'55',
  u'67',
  u'12',
  u'33.6894754264',
  u'-117.543308253'],
 [u'2014-03-15:10:10:20',
  u'MeeToo 1.0',
  u'ef8c7564-0a1a-4650-a655-c8bbd5f8f943',
  u'0',
  u'31',
  u'63',
  u'70',
  u'39',
  u'27',
  u'enabled',
  u'enabled',
  u'enabled',
  u'37.4321088904',
  u'-121.485029632']], "Unexpected entries in the first couple of lines"

#### Working with filteredRDD

Filter out any records which do not parse correctly. The records which do have the correct number of fields should be placed in an RDD named `filteredRDD`.

##### Filtering single line length and taking 1 data

In [0]:
find_length = splitRDD.filter(lambda length: len(length)).take(1)

##### Counting single line length

In [0]:
print(find_length)

[['2014-03-15:10:10:20', 'Sorrento F41L', '8cc3b47e-bd01-4482-b500-28f2342679af', '7', '24', '39', 'enabled', 'disabled', 'connected', '55', '67', '12', '33.6894754264', '-117.543308253']]


##### Condition of accepting RDD data

here we are not going to accept those data which length is not equal to 14

In [0]:
# The format of your answer should be
filteredRDD = splitRDD.filter(lambda length: len(length) == 14)

In [0]:
# DO NOT CHANGE THE CONTENT OF THIS CELL
assert filteredRDD.count() == 459540, "You may have forgotten to account for some delimiters"

#### Working with extractedRDD

Extract the date (first field), model (second field), device ID (third field), latitude and longitude (13th and 14th fields respectively). Each record's list should therefore reduced to a list of the 5 fields only. The output should be placed in a variable named `extractedRDD`.

In [0]:
# The format of your solution should be
extractedRDD = filteredRDD.map(lambda fields: [fields[0],fields[1],fields[2],fields[12],fields[13]])


##### Checking extractedRDD output

In [0]:
extractedRDD.take(2)

Out[19]: [['2014-03-15:10:10:20',
  'Sorrento F41L',
  '8cc3b47e-bd01-4482-b500-28f2342679af',
  '33.6894754264',
  '-117.543308253'],
 ['2014-03-15:10:10:20',
  'MeeToo 1.0',
  'ef8c7564-0a1a-4650-a655-c8bbd5f8f943',
  '37.4321088904',
  '-121.485029632']]

In [0]:
# DO NOT CHANGE THE CONTENT OF THIS CELL
assert extractedRDD.sortBy(lambda x: x[0]).take(2) == [[u'2014-03-15:10:10:20',
  u'Sorrento F41L',
  u'8cc3b47e-bd01-4482-b500-28f2342679af',
  u'33.6894754264',
  u'-117.543308253'],
 [u'2014-03-15:10:10:20',
  u'MeeToo 1.0',
  u'ef8c7564-0a1a-4650-a655-c8bbd5f8f943',
  u'37.4321088904',
  u'-121.485029632']], "There seems to be some discrepancy between your answer and the expected answer"

#### Working with separatedRDD

The second field contains the device manufacturer and model name (e.g. Ronin S2). Split this field by spaces to separate the manufacturer from the model (i.e. manufacturer Ronin, model S2). Each resulting record should therefore now contain a list containing: date, manufacturer, model, device ID, latitude and longitude. The new RDD should be placed in a variable named `separatedRDD`.

In [0]:
# The format of your solution should be
separatedRDD = extractedRDD.map(lambda fields: [fields[0], fields[1].split(' ',1)[0], fields[1].split(' ',1)[1], fields[2], fields[3], fields[4]])

##### Checking separatedRDD output

In [0]:
separatedRDD.take(2)

Out[22]: [['2014-03-15:10:10:20',
  'Sorrento',
  'F41L',
  '8cc3b47e-bd01-4482-b500-28f2342679af',
  '33.6894754264',
  '-117.543308253'],
 ['2014-03-15:10:10:20',
  'MeeToo',
  '1.0',
  'ef8c7564-0a1a-4650-a655-c8bbd5f8f943',
  '37.4321088904',
  '-121.485029632']]

In [0]:
# DO NOT CHANGE THE CONTENT OF THIS CELL
import sys
assert separatedRDD.sortBy(lambda x: x[0]).take(2) == [[u'2014-03-15:10:10:20',
  u'Sorrento',
  u'F41L',
  u'8cc3b47e-bd01-4482-b500-28f2342679af',
  u'33.6894754264',
  u'-117.543308253'],
 [u'2014-03-15:10:10:20',
  u'MeeToo',
  u'1.0',
  u'ef8c7564-0a1a-4650-a655-c8bbd5f8f943',
  u'37.4321088904',
  u'-121.485029632']], "There is some discrepancy between your answer and the expected answer"

# Remove the output directory for the following section if it already exists
if 'dbruntime.dbutils' in sys.modules.keys():
    try:
        dbutils.fs.ls("/FileStore")
        dbutils.fs.ls("/FileStore/devicestatus_etl/")
        dbutils.fs.rm("/FileStore/devicestatus_etl", True)
    except:
        # Directory is not there yet
        pass
else:
    if os.path.exists("/FileStore/devicestatus_etl/"):
        import shutil
        shutil.rmtree("/FileStore/devicestatus_etl/")

#### Saving a text file

Save the extracted data to comma delimited text files in the `/FileStore/devicestatus_etl` directory on DBFS.

In [0]:
extractedRDD = separatedRDD.map(lambda l: l[0] + "," + l[1] + "," + l[2]+ "," + l[3]+ "," + l[4]+ "," + l[5])
extractedRDD.saveAsTextFile("/FileStore/devicestatus_etl")

In [0]:
dbutils.fs.ls("/FileStore/devicestatus_etl")

Out[25]: [FileInfo(path='dbfs:/FileStore/devicestatus_etl/_SUCCESS', name='_SUCCESS', size=0),
 FileInfo(path='dbfs:/FileStore/devicestatus_etl/part-00000', name='part-00000', size=22353655),
 FileInfo(path='dbfs:/FileStore/devicestatus_etl/part-00001', name='part-00001', size=22350220)]

In [0]:
# DO NOT CHANGE THE CONTENT OF THIS CELL
if 'dbruntime.dbutils' in sys.modules.keys():
    assert dbutils.fs.ls("/FileStore/devicestatus_etl"), "You don't appear to have created the required output directory"
    assert dbutils.fs.ls("/FileStore/devicestatus_etl/part-00000"), "The output doesn't appear to be as expected"
    assert dbutils.fs.head("/FileStore/devicestatus_etl/part-00000", 99) == "2014-03-15:10:10:20,Sorrento,F41L,8cc3b47e-bd01-4482-b500-28f2342679af,33.6894754264,-117.543308253", "Expecting different output to that produced"
else:
    assert os.path.exists("/FileStore/devicestatus_etl"), "You don't appear to have created the required output directory"
    assert os.path.exists("/FileStore/devicestatus_etl/part-00000"), "The output doesn't appear to be as expected"
    
    # Check contents of the first line
    with open("/FileStore/devicestatus_etl/part-00000") as f:
        first_line = f.readline().strip()
        assert first_line == "2014-03-15:10:10:20,Sorrento,F41L,8cc3b47e-bd01-4482-b500-28f2342679af,33.6894754264,-117.543308253", "Expecting different output to that produced"
    
# Count the number of lines in the second part
myRDD = sc.textFile("/FileStore/devicestatus_etl/part-00001")
assert myRDD.count() == 229802, "Unexpected number of lines in part-00001"

[Truncated to first 99 bytes]
